In [ ]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/config

In [ ]:
%load_ext streaming_jupyter_integrations.magics

In [ ]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081

In [ ]:
%%flink_execute_sql
CREATE TABLE patient (
  `id` STRING,
  `name` ROW(`family` STRING, `given` STRING ARRAY) ARRAY,
  `gender` STRING,
  `birthDate` STRING,
  `telecom` ROW ( `value` STRING ) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ) ARRAY,
  `maritalStatus` ROW (
    `text` STRING
  )
) WITH (
  'connector' = 'kafka',
  'topic' = 'Patient',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE encounter (
  `id` STRING,
  `status` STRING,
  `type` ROW (
    `coding` ROW (
      `code` STRING,
      `display` STRING
    ) ARRAY  ) ARRAY,
  `subject` ROW (
    `reference` STRING,
    `display` STRING
  ),
  `participant` ROW (
    `individual` ROW (
      `reference` STRING,
      `display` STRING
    )
  ) ARRAY,
  `period` ROW (
    `start` STRING,
    `end` STRING
  ),
  `serviceProvider` ROW (
    `reference` STRING
  ) 
) WITH (
  'connector' = 'kafka',
  'topic' = 'Encounter',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE patient_stats (
  patient_id STRING,
  id_most_seen_doctor STRING,
  id_most_visited_hospital STRING,
  total_doctor_visits INTEGER,
  total_hospital_visits INTEGER
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://ep-dark-sea-a20skpa3-pooler.eu-central-1.aws.neon.tech:5432/verceldb',
  'table-name' = 'public.patient_stats',
  'username' = 'default',
  'password' = 'tUyXvsaMq07d'
);

In [ ]:
%%flink_execute_sql
CREATE VIEW doctor_visits AS
SELECT
  e.subject.reference AS patient_id,
  e.participant[1].individual.reference AS doctor_id,
  COUNT(*) AS visit_count
FROM
  encounter e
GROUP BY
  e.subject.reference,
  e.participant[1].individual.reference;

In [ ]:
%%flink_execute_sql
CREATE VIEW hospital_visits AS
SELECT
  e.subject.reference AS patient_id,
  e.serviceProvider.reference AS hospital_id,
  COUNT(*) AS visit_count
FROM
  encounter e
GROUP BY
  e.subject.reference,
  e.serviceProvider.reference;

In [ ]:
%%flink_execute_sql
CREATE VIEW most_seen_doctors AS
SELECT
  patient_id,
  doctor_id AS id_most_seen_doctor,
  MAX(visit_count) AS total_doctor_visits
FROM (
  SELECT
    patient_id,
    doctor_id,
    visit_count,
    ROW_NUMBER() OVER (PARTITION BY patient_id ORDER BY visit_count DESC) AS rn
  FROM doctor_visits
)
WHERE rn = 1
GROUP BY patient_id, doctor_id;

In [ ]:
%%flink_execute_sql
CREATE VIEW most_visited_hospitals AS
SELECT
  patient_id,
  hospital_id AS id_most_visited_hospital,
  MAX(visit_count) AS total_hospital_visits
FROM (
  SELECT
    patient_id,
    hospital_id,
    visit_count,
    ROW_NUMBER() OVER (PARTITION BY patient_id ORDER BY visit_count DESC) AS rn
  FROM hospital_visits
)
WHERE rn = 1
GROUP BY patient_id, hospital_id;

In [ ]:
%%flink_execute_sql
SELECT
  p.id AS patient_id,
  d.id_most_seen_doctor,
  d.total_doctor_visits,
  h.id_most_visited_hospital,
  h.total_hospital_visits
FROM
  patient p
LEFT JOIN
  most_seen_doctors d
ON p.id = d.patient_id
LEFT JOIN
  most_visited_hospitals h
ON p.id = h.patient_id;


In [23]:
%%flink_execute_sql
INSERT INTO patient_stats
WITH most_seen_doctor AS (
  SELECT
    e.`subject`.`reference` AS patient_id,
    COUNT(*) AS total_doctor_visits,
    MAX(e.`participant`[1].`individual`.`reference`) AS id_most_seen_doctor
  FROM encounter e
  JOIN patient p ON e.`subject`.`reference` = p.`id`
  GROUP BY e.`subject`.`reference`
),
most_visited_hospital AS (
  SELECT
    e.`subject`.`reference` AS patient_id,
    COUNT(*) AS total_hospital_visits,
    MAX(e.`serviceProvider`.`reference`) AS id_most_visited_hospital
  FROM encounter e
  JOIN patient p ON e.`subject`.`reference` = p.`id`
  GROUP BY e.`subject`.`reference`
)
SELECT
  p.`id` AS patient_id,
  COUNT(DISTINCT e.`serviceProvider`.`reference`) AS total_patient_count,
  COUNT(*) AS total_visit_count,
  MAX(msd.`id_most_seen_doctor`) AS id_most_seen_doctor,
  MAX(msd.`total_doctor_visits`) AS total_doctor_visits,
  MAX(mvh.`id_most_visited_hospital`) AS id_most_visited_hospital,
  MAX(mvh.`total_hospital_visits`) AS total_hospital_visits
FROM encounter e
JOIN patient p ON e.`subject`.`reference` = p.`id`
LEFT JOIN most_seen_doctor msd ON e.`subject`.`reference` = msd.`patient_id`
LEFT JOIN most_visited_hospital mvh ON e.`subject`.`reference` = mvh.`patient_id`
GROUP BY p.`id`;


This job runs in a background, please either wait or interrupt its execution before continuing


Job starting...
Execution failed


An error occurred while calling o32.executeSql.
: org.apache.flink.table.api.ValidationException: SQL validation failed. SQL validation failed. From line 24, column 3 to line 24, column 27: Expression 'msd.id_most_seen_doctor' is not being grouped
	at org.apache.flink.table.planner.calcite.FlinkPlannerImpl.org$apache$flink$table$planner$calcite$FlinkPlannerImpl$$validate(FlinkPlannerImpl.scala:200)
	at org.apache.flink.table.planner.calcite.FlinkPlannerImpl.validate(FlinkPlannerImpl.scala:117)
	at org.apache.flink.table.planner.operations.SqlNodeToOperationConversion.convert(SqlNodeToOperationConversion.java:261)
	at org.apache.flink.table.planner.delegation.ParserImpl.parse(ParserImpl.java:106)
	at org.apache.flink.table.api.internal.TableEnvironmentImpl.executeSql(TableEnvironmentImpl.java:728)
	at jdk.internal.reflect.GeneratedMethodAccessor18.invoke(Unknown Source)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.